## Camera Calibration

This Notebook contains codes to calibrate the camera using images in the folder "chessboard".

In [ ]:
#!/usr/bin/env python

import cv2
import numpy as np
import os
import glob
import ipywidgets
from IPython.display import display

In [2]:
# Defining the dimensions of checkerboard
CHECKERBOARD = (6,8)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [3]:
# create and display image widget
image_widget = ipywidgets.Image(format='jpeg')
display(image_widget)

# Extracting path of individual image stored in a given directory
images = glob.glob('./chessboard/*.jpg')

i = 0
for fname in images:
    
    i += 1
    img = cv2.imread(fname)
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    
    if ret == True:
        
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        
        imgpoints.append(corners2)

        # Draw and display the corners    
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        
        output_fname = 'cornerfind'+ str(i) + '.jpg'
        cv2.imwrite(output_fname, img)

        # set value of image widget (must compress to JPEG to send over web).  When this value is set, the data is transferred to desktop machine and image is rendered
        image_widget.value = bytes(cv2.imencode('.jpg', img)[1])
    else:
        print("No detection.")

cv2.destroyAllWindows()


Image(value=b'', format='jpeg')

No detection.
No detection.
No detection.
No detection.
No detection.
No detection.
No detection.
No detection.
No detection.


In [4]:
h,w = img.shape[:2]

"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Retval : \n")
print(ret)
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)


Retval : 

3.217901008090897
Camera matrix : 

[[141.95965107   0.         148.48829074]
 [  0.         183.45309821 167.19170579]
 [  0.           0.           1.        ]]
dist : 

[[-0.33845104  0.13242484 -0.00241765 -0.00145509 -0.02609716]]
rvecs : 

[array([[-0.48197212],
       [-0.538951  ],
       [ 1.53860515]]), array([[ 0.03272844],
       [-0.01304433],
       [-1.49193289]]), array([[-0.34503838],
       [ 0.01686722],
       [ 1.45002293]]), array([[-0.71961769],
       [-0.6298202 ],
       [ 1.45466852]]), array([[-0.04997721],
       [-0.30446041],
       [ 1.53891488]]), array([[-0.7701713 ],
       [ 0.10974813],
       [ 1.36992417]]), array([[0.36708846],
       [0.02216793],
       [0.98269925]]), array([[-0.14425594],
       [-0.31447982],
       [ 1.58920506]]), array([[-0.30694296],
       [ 0.36340847],
       [-1.5137487 ]]), array([[0.58166448],
       [0.38097513],
       [1.64271806]]), array([[0.53863889],
       [0.45377372],
       [1.50055327]]), arr

In [8]:
# Extracting path of individual image stored in a given directory
images = glob.glob('./chessboard/*.jpg')
i = 0
for fname in images:
    img = cv2.imread(fname)
    i += 1
    
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    
    output_fname = './undistort/undistort'+ str(i) + '.jpg'
    cv2.imwrite(output_fname, dst)
    
cv2.destroyAllWindows()




In [11]:
import pickle

filename = "calibration.dat"

filehandler = open(filename, 'wb')

calibration = {"mtx": mtx,
    "dist": dist,
    "rvecs" : rvecs,
    "tvecs" : tvecs}

pickle.dump(calibration, filehandler)
